# 建立所有商品名稱 + 商品分類細項 的list

### 1. pd.read_csv('自己的檔案路徑')

In [2]:
import pandas as pd
df = pd.read_csv('./dataset/keywords/keywords_LA1.csv')
df_dict = df.set_index('product_name').to_dict()
items_dict = df_dict['aisle']

print(len(items_dict))
#print(items_dict)

8200


# 取得Product info Dict

### 2. with open('product_info_自己負責的區.json', 'w') as fp:
### 3. win的user 請改 soup = BeautifulSoup(req.text, "lxml")
### 4. with open('product_info_LA1.json', 'a', encoding='utf-8') as fp:因為改為單筆寫入 舊的要記得刪掉

In [3]:
def getInfo(search_result):
    for item_info in search_result:

        product_info = {
            "keyword": "",
            "name": "",
            "brand": "",
            "url": "",
            "pic": "",
            "price": "",
            "category": [],
            "star_ratings": [],
            "at_a_glance": [],
            "highlights": [],
            "specifications": {},
            "description": "",
            "reviews": []
        }
        
        if (item_info.get('type')=='Collection Parent'):
            continue
            
        p_name = item_info.get('title', 'NA')
        p_brand = item_info.get('brand', 'NA')
        p_price = lambda x:x['price'].get('formatted_current_price', 'NA') if item_info.get('price') else 'NA'
        p_glances = item_info.get("wellness_merchandise_attributes", 'NA')
        p_description = item_info.get('description', 'NA')

        #rating
        p_ratings_s_avg = item_info.get('average_rating', 'NA')
        p_ratings_s_cnt = item_info.get('total_reviews', 'NA')

        p_url = lambda x: 'https://www.target.com'+x if item_info.get('url') else 'NA'
        p_pic = lambda x: x.get('base_url')+x.get('primary') if item_info.get('images') else 'NA'
        p_highlights = lambda x:x['soft_bullets'].get('bullets', 'NA') if item_info.get('soft_bullets') else 'NA' 

        #category
        if item_info.get('url'):
            try:
                req = requests.get(p_url(item_info['url']))
                #win的user 請改 soup = BeautifulSoup(req.text, "lxml")
                soup = BeautifulSoup(req.text, "html")
                p_category = soup.find("div", {"class", "h-text-sm h-padding-v-tiny"}).text.strip("\u200e").split("/")
                product_info['category'] = p_category[1:]
            except:
                product_info['category'] = 'NA'
        else:
            product_info['category'] = 'NA'

        #reviews
        p_review = item_info.get("top_reviews", 'NA')

        product_info['keyword'] = item
        product_info['name'] = p_name
        product_info['url'] = p_url(item_info['url'])
        product_info['brand'] = p_brand
        product_info['pic'] = p_pic(item_info['images'][0])
        product_info['price'] = p_price(item_info)
        product_info['star_ratings'].append(p_ratings_s_avg) 
        product_info['star_ratings'].append(p_ratings_s_cnt) 

        #specifications
        spec_list = ["Contains", "Form", "State of Readiness", "Store", "Package Quantity", "Package type", "Net weight"]
        if item_info.get("bullet_description"):
            match=0
            for i in spec_list:
                for j in item_info["bullet_description"]:
                    j = re.sub("<.*?>", " ",j).strip(' ')
                    if re.match(i, j.split(": ")[0]):
                        product_info["specifications"][i] = j.split(": ")[1]
                        match+=1
            if(match==0):
                product_info["specifications"]='NA'
        else:
            product_info["specifications"]='NA'

        #description
        product_info['description'] = p_description
        product_info['description'] = re.sub("<.*?>", " ",product_info['description'])

        #reviews
        if (p_review == "NA"):
            product_info["reviews"] = "NA"
        else:
            for rev in p_review:
                review = re.sub('[\r\n\t]', '', rev["review_text"])
                product_info["reviews"].append(review)

        #at_a_glance
        if (p_glances=='NA'):
            product_info['at_a_glance']= 'NA'
        else:
            for glance in p_glances:
                product_info['at_a_glance'].append(glance["value_name"])

        #highlight
        if (p_highlights(item_info)=='NA'):
            product_info['highlights'] = 'NA'
        else:
            for highlight in p_highlights(item_info):
                product_info['highlights'].append(highlight)

        #products.append(product_info)
        #print(product_info)
        with open('product_info_LA1.json', 'a', encoding='utf-8') as fp:
            global count
            count+=1

            if (count==1):
                fp.write('[')
                json.dump(product_info, fp, indent=4, ensure_ascii=False)
                fp.write(',')
            elif(count==8200):
                json.dump(product_info, fp, indent=4, ensure_ascii=False)
                fp.write(']')
            else:
                json.dump(product_info, fp, indent=4, ensure_ascii=False)
                fp.write(',')

            print(f'{count} finished.')

       

# 解析網址 + 更改參數 get info

### 4. url = '自己的地點的url' (count=6)

In [ ]:
import requests
import json
from bs4 import BeautifulSoup
import re
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse

url = 'https://redsky.target.com/v2/plp/search/?channel=web&count=6&default_purchasability_filter=true&facet_recovery=false&fulfillment_test_mode=grocery_opu_team_member_test&isDLP=false&keyword=Peanut+Butter+Cereal&offset=0&pageId=%2Fs%2FPeanut+Butter+Cereal&pricing_store_id=1306&store_ids=1306%2C3294%2C198%2C2775%2C2632&visitorId=0172C7C1A02802019D4E379FB6434C3C&include_sponsored_search_v2=true&ppatok=AOxT33a&platform=mobile&useragent=Mozilla%2F5.0+%28Linux%3B+Android+6.0%3B+Nexus+5+Build%2FMRA58N%29+AppleWebKit%2F537.36+%28KHTML%2C+like+Gecko%29+Chrome%2F83.0.4103.116+Mobile+Safari%2F537.36&excludes=available_to_promise_qualitative%2Cavailable_to_promise_location_qualitative&key=eb2551e4accc14f38cc42d32fbc2b2ea'
item=''
search_item =''
count = 0
#products=[]
no_result = []

for key, value in items_dict.items():
    
    item = key
    search_item = str(key) + " " + str(value)
    
    url = urlparse(url)
    list_url = list(url)
    #set new params
    params = dict( (k, v if len(v)>1 else v[0] ) for k, v in parse_qs(url.query).items() )
    params['include_sponsored_search_v2'] = 'false'
    params['keyword'] = search_item
    params['pageId'] = '/s/%s'%(search_item)


    #update url
    list_url[4] = urlencode(params)
    url = urlunparse(list_url)
    item = item
    search_item = search_item 
    print(search_item, url)
    
    #get url res
    useragent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
    headers = {'User-Agent':useragent}
    res = requests.get(url, headers=headers, verify=False)
    
    #check status code
    if (res.status_code==200):
        json_str = res.text
        js = json.loads(json_str)
        js_list = list(js.values())
        
        search_result = js_list[1]['items']['Item']
        print(search_result)
        getInfo(search_result)
        
    else:
        no_result.append(item)
        print('%s : no result'%(item))
        


3 in 1 Cedar & Citrus Lotion for Every Man soap https://redsky.target.com/v2/plp/search/?channel=web&count=6&default_purchasability_filter=true&facet_recovery=false&fulfillment_test_mode=grocery_opu_team_member_test&isDLP=false&keyword=3+in+1+Cedar+%26+Citrus+Lotion+for+Every+Man+soap&offset=0&pageId=%2Fs%2F3+in+1+Cedar+%26+Citrus+Lotion+for+Every+Man+soap&pricing_store_id=1306&store_ids=1306%2C3294%2C198%2C2775%2C2632&visitorId=0172C7C1A02802019D4E379FB6434C3C&include_sponsored_search_v2=false&ppatok=AOxT33a&platform=mobile&useragent=Mozilla%2F5.0+%28Linux%3B+Android+6.0%3B+Nexus+5+Build%2FMRA58N%29+AppleWebKit%2F537.36+%28KHTML%2C+like+Gecko%29+Chrome%2F83.0.4103.116+Mobile+Safari%2F537.36&excludes=available_to_promise_qualitative%2Cavailable_to_promise_location_qualitative&key=eb2551e4accc14f38cc42d32fbc2b2ea


/opt/anaconda3/envs/python_2020/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'redsky.target.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


[{'subscribable': False, 'package_dimensions': {'weight': '1.22', 'weight_unit_of_measure': 'POUND', 'width': '3.27', 'depth': '7.87', 'height': '1.64', 'dimension_unit_of_measure': 'INCH'}, 'title': 'Every Man Jack Citrus Body Scrub - 16.9oz', 'tcin': '13406707', 'type': 'Stand Alone', 'dpci': '049-00-2126', 'upc': '878639000063', 'url': '/p/every-man-jack-citrus-body-scrub-16-9oz/-/A-13406707', 'description': 'Be good to your body, naturally, with Every Man Jack Citrus Body Scrub. This citrus scented body scrub and shower gel is formulated with biodegradable jojoba beads to quickly scrub away dry and dead skin leaving the body clean and clear. Essential citrus oils of mandarin orange and lemon create a lasting citrus scent. Perfect for everyday use and all skin types. ', 'representative_child_part_number': '13406707', 'merch_sub_class': '0', 'merch_class': 'PERSONAL CARE', 'merch_class_id': '49', 'brand': 'Every Man Jack', 'product_brand': {'facet_id': '5td0z', 'brand': 'Every Man Ja

1 finished.
2 finished.
3 finished.
4 finished.
5 finished.
